# **Contour detection in webcam by using TrackBars IN Windows:**

In [ ]:

import cv2
import numpy as np

#Read Camera
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
def nothing(x):
    pass
#window name
cv2.namedWindow("Color Adjustments",cv2.WINDOW_NORMAL)
cv2.resizeWindow("Color Adjustments", (300, 300))
cv2.createTrackbar("Thresh", "Color Adjustments", 0, 255, nothing)

#COlor Detection Track

cv2.createTrackbar("Lower_H", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_S", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_V", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Upper_H", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_S", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_V", "Color Adjustments", 255, 255, nothing)


while True:
    _,frame = cap.read()
    frame = cv2.resize(frame,(400,400))
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #detecting hand
    l_h = cv2.getTrackbarPos("Lower_H", "Color Adjustments")
    l_s = cv2.getTrackbarPos("Lower_S", "Color Adjustments")
    l_v = cv2.getTrackbarPos("Lower_V", "Color Adjustments")

    u_h = cv2.getTrackbarPos("Upper_H", "Color Adjustments")
    u_s = cv2.getTrackbarPos("Upper_S", "Color Adjustments")
    u_v = cv2.getTrackbarPos("Upper_V", "Color Adjustments")

    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])

    #Creating Mask
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    #filter mask with image
    filtr = cv2.bitwise_and(frame, frame, mask=mask)


    mask1  = cv2.bitwise_not(mask)
    m_g = cv2.getTrackbarPos("Thresh", "Color Adjustments") #getting track bar value
    ret,thresh = cv2.threshold(mask1,m_g,255,cv2.THRESH_BINARY)
    dilata = cv2.dilate(thresh,(1,1),iterations = 6)



    #findcontour(img,contour_retrival_mode,method)
    cnts,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #Here cnts is a list of contours. ANd each contour is an array with x, y cordinate
    #hier variable called hierarchy and it contain image information.
    #print("Number of contour==",cnts,"\ntotal contour==",len(cnts))
    #print("Hierarchy==\n",hier)

    #Draw the contours
    #frame= cv2.drawContours(frame,cnts,-1,(176,10,15),4)

    # loop over the contours

    for c in cnts:
        epsilon = 0.0001*cv2.arcLength(c,True)
        data= cv2.approxPolyDP(c,epsilon,True)

        hull = cv2.convexHull(data)
        cv2.drawContours(frame, [c], -1, (50, 50, 150), 2)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)
        """
        hull = cv2.convexHull(data,returnPoints = False)
        defect = cv2.convexityDefects(data[0],hull)
        print("defect==",defect)
        """
    cv2.imshow("Thresh", thresh)
    cv2.imshow("mask==",mask)
    cv2.imshow("filter==",filtr)
    cv2.imshow("Result", frame)

    key = cv2.waitKey(25) &0xFF
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()




# **Contour detection by using Image Path in Colab.**

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, IntSlider
from ipywidgets import fixed


# Load and verify an image
def load_image(path):
    image = cv2.imread(path)
    if image is None:
        print("Error: Could not load image. Check the path.")
    return image

# Display function for side-by-side images
def display_images(images, titles):
    fig, axs = plt.subplots(1, len(images), figsize=(15, 5))
    for ax, img, title in zip(axs, images, titles):
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_title(title)
        ax.axis("off")
    plt.show()

# Image processing with HSV filtering and contour display
def process_image(lower_h, lower_s, lower_v, upper_h, upper_s, upper_v, thresh, frame):
    frame_resized = cv2.resize(frame, (400, 400))
    hsv = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2HSV)

    # Define mask boundaries
    lower_bound = np.array([lower_h, lower_s, lower_v])
    upper_bound = np.array([upper_h, upper_s, upper_v])

    # Create mask and filter the image
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    filtr = cv2.bitwise_and(frame_resized, frame_resized, mask=mask)

    # Apply thresholding and dilate for contour visibility
    _, thresh_img = cv2.threshold(mask, thresh, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh_img, None, iterations=3)

    # Find contours and draw them
    cnts, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in cnts:
        cv2.drawContours(frame_resized, [c], -1, (0, 255, 0), 2)

    # Display images in one row
    display_images([thresh_img, mask, filtr, frame_resized],
                   ["Threshold", "Mask", "Filtered", "Result"])

# Specify the image path
image_path = "/content/hand1.jpeg"
frame = load_image(image_path)

if frame is None:
    print("Exiting program due to failed image load.")
else:
    # Interactive sliders with frame as a fixed argument
    interact(process_image,
             lower_h=IntSlider(0, 0, 180),
             lower_s=IntSlider(0, 0, 255),
             lower_v=IntSlider(0, 0, 255),
             upper_h=IntSlider(150, 0, 180),
             upper_s=IntSlider(255, 0, 255),
             upper_v=IntSlider(255, 0, 255),
             thresh=IntSlider(128, 0, 255),
             frame=fixed(frame))  # Pass 'frame' as a fixed parameter

interactive(children=(IntSlider(value=0, description='lower_h', max=180), IntSlider(value=0, description='lowe…